<a href="https://colab.research.google.com/github/torresmateo/penguin-tf-workshop/blob/master/D4_3_Pronostico_multivariable.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Pronóstico multivariable

En este notebook, extendemos el poder predictivo de nuestro modelo de series temporales incluyendo más variables en el 
entrenamiento.

Primero que nada, importamos las bibliotecas necesarias

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
# importamos Pandas, que nos permitirá ver mejor los datos
import pandas as pd

# configuramos las figuras de matplotlib
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
plt.style.use('default')

Bajamos los datos, y definimos la función que nos ayudará a visualizar las predicciones

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)
df = pd.read_csv(csv_path)
df.head()

def ver_ejemplo(ejemplo, delta, titulos):
    rotulos = ['Historia', 'Futuro verdadero'] + titulos[1:]
    marker = ['.-', 'rx', 'go', 'mD', 'c<']
    tiempos = list(range(-ejemplo[0].shape[0], 0))
    if delta:
        futuro = delta
    else:
        futuro = 0
    
    plt.title(titulos[0])
    for i, x in enumerate(ejemplo):
        if i:  # futuro verdadero o predicción
            plt.plot(futuro, ejemplo[i], marker[i], markersize=10, label=rotulos[i])
        else:  # historia
            plt.plot(tiempos, ejemplo[i], marker[i], label=rotulos[i])
    plt.legend()
    plt.xlim([tiempos[0], (futuro+5)*2])
    plt.xlabel('Tiempo')
    return plt

def ver_ejemplo_multivalor(historia, futuro_real, prediccion):
    plt.figure(figsize=(12, 6))
    tiempos_historia = list(range(-len(historia), 0))
    futuro = len(futuro_real)

    plt.plot(tiempos_historia, np.array(historia[:, 1]), label='Historia')
    plt.plot(np.arange(futuro)/6, np.array(futuro_real), 'bo',
            label='Futuro Real')
    if prediccion.any():
        plt.plot(np.arange(futuro)/6, np.array(prediccion), 'ro',
                label='Prediccion')
    plt.legend(loc='upper left')
    plt.show()


# Preparar un *dataset* multivariable

Si bien el dataset original es de 14 variables, por simplicidad consideremos 3:
* temperatura
* presión atmosférica
* densidad del aire

In [ ]:
lista_features = ['p (mbar)', 'T (degC)', 'rho (g/m**3)']

filtramos el *dataset*, dejando solamente las columnas correspondientes a nuestra lista de features

In [ ]:
features = df[lista_features]
features.index = df['Date Time']
features.head()

exploremos los datos

In [ ]:
features.plot(subplots=True)

# Particion y normalización del *dataset*

Al igual que el caso univariable, debemos normalizar solo considerando la media y desviación estándar del *training set*

In [ ]:
particion = 300000

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# por ahora solo extraemos el training set, que se usa para calcular los parámetros para normalizar
training_data = features[:particion] 
scaler.fit(training_data)

# normalizamos todo el dataset
features = scaler.transform(features)

# Preparamos los ejemplos para una predicción multivariable

Esta función sirve para procesar el *dataset* y generar ejemplos con los *inputs* y *labels* que luego se usarán para el entrenamiento y validación de nuestro modelo. De forma similar al ejemplo univariale, consideraremos una ventana del pasado para predecir un punto en el futuro. Sin embargo, también incluimos la opción de usar varios puntos futuros como *label*.

In [ ]:
def ejemplos_multivariable(dataset, target, inicio, fin, ancho_ventana, offset_prediccion, paso, multi_valor = False):
    data = []
    labels = []

    inicio = inicio + ancho_ventana
    if fin is None:
        fin = len(dataset) - offset_prediccion
    
    for i in range(inicio, fin):
        # el paso nos permite tomar diferentes resoluciones de muestra.
        # En este caso, el dataset tiene una muestra cada 10 minutos.
        # Si quisieramos, por ejemplo, una muestra por hora, el paso es 6.
        # Definimos nuestra ventana tomando eso en cuenta
        ventana = range(i-ancho_ventana, i, paso)
        
        # agregamos la muestras a nuestro ejemplo
        data.append(dataset[ventana])

        # agregamos el label
        if multi_valor:
            # en caso de que queramos predecir muchos valores, agregamos todas
            # las muestras que estan después de la ventana, hasta el límite de
            # predicción
            labels.append(target[i:i+offset_prediccion])
        else:
            # si predecimos un solo valor, hacemos lo mismo que en el ejemplo anterior, y 
            # solo agregamos la muestra en el indice ubicado `prediccion` lugares luego de 
            # la ventana
            labels.append(target[i+offset_prediccion])
    
    return np.array(data), np.array(labels)

con esa función, podemos hacer la partición de nuestro dataset en *training* y *testing*. Queremos que nuestro modelo vea datos de los útimos 5 días, es decir 720 muestras (5 x 24 x 6) y queremos predecir la temperatura 12 horas en el futuro, luego de 72 muestras (12 x 6).

El `target` es la temperatura, y por ello le pasamos solo la segunda columna del dataset (`features[:,1]`). También cambiamos la resolución de las muestras a una muestra por hora, por lo que establecemos el `paso` a 6.

In [ ]:
x_train, y_train = ejemplos_multivariable(features, features[:, 1], 0, particion, 720, 72, 6)
x_test, y_test = ejemplos_multivariable(features, features[:, 1], particion, None, 720, 72, 6)

Igual al ejemplo anterior, preparamos el dataset usando `tf.data` partiendo el dataset en *batches* con *shuffling*.

In [ ]:
batch = 256
buffer = 10000

# creamos el training dataset
train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train = train.cache().shuffle(buffer).batch(batch).repeat()

# creamos el testing dataset
test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test = test.batch(batch).repeat()

# Creamos la red neuronal

Podemos usar la misma red que en el caso anterior. Observar que la dimensionalidad de nuestros datos es diferente, por lo que automáticamente el *layer* con unidades LSTM se adaptará cuando usamos `shape[-2:]`

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.LSTM(8, input_shape=x_train.shape[-2:]),
                                    tf.keras.layers.Dense(1)])

model.compile(optimizer='adam', loss='mae')

entrenamos

In [ ]:
model.fit(train, epochs=10, steps_per_epoch=200, validation_data=test, validation_steps=50)

y vemos unas cuantas predicciones

In [ ]:
for x,y in test.take(4):
    ver_ejemplo([x[0][:,1].numpy() ,y[0].numpy(), model.predict(x)[0]], 12, 
                ['Comparación de modelos', 'shallow LSTM'])
    plt.show()

# Extrapolación de muchos valores

En este caso, queremos usar la misma información de entrada, pero en lugar de predecir la temperatura luego de 12 horas, queremos predecir la temperatura para cada una de las siguientes 12 horas.

Los parámetros son similares, con la única diferencia que necesitamos es indicarle a nuestro generador de ejemplos que los *labels* son multi valor.

In [ ]:
x_train_multi, y_train_multi = ejemplos_multivariable(features, features[:, 1], 0, particion, 720, 72, 6, multi_valor=True)
x_test_multi, y_test_multi = ejemplos_multivariable(features, features[:, 1], particion, None, 720, 72, 6, multi_valor=True)

In [ ]:
x_train_multi.shape, y_train_multi.shape

y lo pre-procesamos con `tf.data`

In [ ]:
batch = 256
buffer = 10000

# creamos el training dataset
train_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_multi = train_multi.cache().shuffle(buffer).batch(batch).repeat()

# creamos el testing dataset
test_multi = tf.data.Dataset.from_tensor_slices((x_test_multi, y_test_multi))
test_multi = test_multi.batch(batch).repeat()

veamos un ejemplo del dataset

In [ ]:

for x, y in train_multi.take(1):
    ver_ejemplo_multivalor(x[0], y[0], np.array([0]))

# Defninimos la red neuronal



In [ ]:
model_multi = tf.keras.models.Sequential([tf.keras.layers.LSTM(32, return_sequences=True, input_shape=x_train.shape[-2:]),
                                          tf.keras.layers.LSTM(16, activation='relu'),
                                          tf.keras.layers.Dense(72)])

model_multi.compile(optimizer='adam', loss='mae')

entrenamos

In [ ]:
model_multi.fit(train, epochs=10, steps_per_epoch=200, validation_data=test, validation_steps=50)

vemos unas cuantas predicciones

In [ ]:
for x, y in test_multi.take(3):
    ver_ejemplo_multivalor(x[0], y[0], model_multi.predict(x)[0])

# Créditos

Este notebook traduce y adapta el código y explicaciones del [Tutorial de Tensorflow](https://www.tensorflow.org/tutorials/structured_data/time_series) en series temporales.